In [1]:
from bs4 import BeautifulSoup
import requests
import re
import lxml.html as lh
from urllib.request import urlretrieve
import pandas as pd
import urllib
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import textract

def is_course_code(c):
    try:
        return c[0].isupper() and c[1].isupper() and c[2].isupper() and c[3].isupper()
    except:
        return False

def parse_prereq(s):
    if(s==""):
        return ""
    requisite =""
    for word in s.split():
        if(is_course_code(word)):
            requisite =requisite+ word.replace(".","")
            
        if('.' in word):
            word=word.replace('.',"")
            if(len(requisite)>0):
                requisite=requisite+")"
                requisite=requisite+" AND ("
                
        if(word=='not' or word.lower()=='incompatible'):
            requisite=requisite+" NOT ("
        if(word=='and'):
            requisite=requisite+" AND "
        if(word=='or'):
            requisite=requisite+" OR "
    if(s[-1]=='.'):    
        requisite=requisite[:len(requisite)-len( " AND (")]
    if(len(requisite)==0):
        return ""
    if(not(requisite[-1]==")")):
        requisite=requisite+")"
    return requisite

In [ ]:
f = open('course_list','r')
final_course_list=[]
content = f.readlines()
for c in content:
    if(c[0].isupper() and c[1].isupper() and c[2].isupper()):
        final_course_list.append(c[0:8])

pd.DataFrame(final_course_list).to_csv("final_course_list.csv")

f.close()
#f = open('course_list','w')

In [18]:
course_codes = list(pd.read_csv("courses.csv")['\n                    Code\n                '])
course_names = list(pd.read_csv("courses.csv")['\n                    Title\n                '])
course_descriptions=[]
course_codes=final_course_list
for i in range(0,len(final_course_list)):
    if(course_codes[i] in seen):
        break
    seen.add(i)
    course=course_codes[i]
    print("Downloading reports for:",course)
    main_page = requests.get("https://programsandcourses.anu.edu.au/2018/course/"+course)
    soup = BeautifulSoup(main_page.content, 'html.parser')
    desc=""
    req=""
    outcomes=""
    req_description=""
    full_text=soup.get_text()
    for s in soup.find_all("div","introduction"):
        desc = s.get_text()
    for s in soup.find_all("div","requisite"):
        req=parse_prereq(s.get_text())
        
        req_description=s.get_text()
        
    for s in soup.find_all("ol"):
        outcomes = s.get_text()    

   
    
    course_descriptions.append([course,course_names[i],desc,outcomes,req,req_description,full_text])


ConnectionError: HTTPSConnectionPool(host='programsandcourses.anu.edu.au', port=443): Max retries exceeded with url: /2018/course/PASI2030 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x118ea4208>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [19]:
df=pd.DataFrame(course_descriptions)

In [20]:
df.to_csv("course_descriptions.csv")

In [17]:
print(df)

          0                                          1  \
0  ENGL6020    16th, 17th and 18th  Century Literature   
1  ENGL3005  16th, 17th and 18th Century Literature(H)   
2  PREP1102                       20th Century History   

                                                   2  \
0  A study of selected poetry, prose and drama fr...   
1  A study of selected poetry, prose and drama fr...   
2  This course examines major global events such ...   

                                                   3  \
0  Apply knowledge of the historical and cultural...   
1  Apply knowledge of the historical and cultural...   
2                                                      

                                                   4  
0  \n\n\n\n\n16th, 17th and 18th  Century Literat...  
1  \n\n\n\n\n16th, 17th and 18th Century Literatu...  
2  \n\n\n\n\n20th Century History - ANU\n\n\n\n\n...  


In [51]:
from py2neo import Graph, Node, Relationship
#"http://localhost:7200/repositories/CourseAIOntology")
import rdflib


g=rdflib.Graph()
g.load('http://localhost:7200/repositories/CourseAIOntology')

HTTPError: HTTP Error 400: Bad Request